In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from pprint import pprint
from data_process import build_vocab, batch_iter, sentence_to_index
from models import RNN

# RNN Operation: $h_t=tanh(W*[x_t;h_{t-1}] + b)$
<img src="https://cdn-images-1.medium.com/max/1600/1*NKhwsOYNUT5xU7Pyf6Znhg.png">

# RNN Operation using numpy

In [ ]:
batch_size = 1
time_step = 10
input_size = 4
hidden_size = 3

In [ ]:
x_input = np.random.normal(size=(batch_size, input_size))
prev_state = np.random.normal(size=(batch_size, hidden_size))
w = np.random.normal(size=(input_size + hidden_size, hidden_size))
b = np.random.normal(size=(hidden_size))
print('x_input:')
pprint(x_input)
print('\nprev_state:')
pprint(prev_state)

In [ ]:
def RNN_numpy(x_input, previous_state, w, b):
    x = np.concatenate((x_input, previous_state), axis=1)
    state = np.tanh(np.add(np.matmul(x, w), b))
    output = state
    return output, state

In [ ]:
output, state = RNN_numpy(x_input, prev_state, w, b)
print('output:')
pprint(output)
print('\nstate:')
pprint(state)

# RNN Operation using tensorflow

In [ ]:
tf.reset_default_graph()
x_input_tensor = tf.constant(x_input, dtype=tf.float32)
prev_state_tensor = tf.constant(prev_state, dtype=tf.float32)

In [ ]:
cell = tf.nn.rnn_cell.BasicRNNCell(hidden_size, reuse=tf.AUTO_REUSE)
cell(x_input_tensor, prev_state_tensor)
w_assign = tf.assign(cell.trainable_variables[0], w)
b_assign = tf.assign(cell.trainable_variables[1], b)
sess = tf.Session()
sess.run([w_assign, b_assign])
print('x_input_tensor:')
pprint(sess.run(x_input_tensor))
print('\nprev_state_tensor:')
pprint(sess.run(prev_state_tensor))

In [ ]:
output_tensor, state_tensor = cell(inputs=x_input_tensor, state=prev_state_tensor)
print('output:')
pprint(sess.run(output_tensor))
print('\nstate:')
pprint(sess.run(state_tensor))

# RNN Sequence Operation using numpy 

In [ ]:
x_seq_input = np.random.normal(size=(batch_size, time_step, input_size))
initial_state = np.random.normal(size=(batch_size, hidden_size))
print('x_seq_input:')
pprint(x_seq_input)
print('\ninitial_state:')
pprint(initial_state)

In [ ]:
def seqRNN(x_input, initial_state, w, b):
    state = initial_state
    output = []
    for i in range(x_input.shape[1]):
        _, state = RNN_numpy([x_input[0][i]], state, w, b)
        output.append(state)
    return output, state

In [ ]:
output, state = seqRNN(x_seq_input, initial_state, w ,b)

In [ ]:
print('output:')
pprint(output)
print('\nstate:')
pprint(state)

# RNN Sequence Operation with tensorflow

In [ ]:
x_seq_input_tensor = tf.constant(x_seq_input, dtype=tf.float32)
initial_state_tensor = tf.constant(initial_state, dtype=tf.float32)

In [ ]:
output, state = tf.nn.dynamic_rnn(cell, x_seq_input_tensor, 
                                  initial_state=initial_state_tensor,
                                  dtype=tf.float32)

In [ ]:
print('output:')
pprint(sess.run(output))
print('\nstate:')
pprint(sess.run(state))

# from text to RNN Sequence Operation with tensorflow

In [ ]:
data = ['나는 생각한다 고로 나는 존재한다.',
        '모든 국가는 그에 걸맞는 국가를 가진다.',
        '이것 또한 지나가리라',
        '죄는 미워하되 사람은 미워하지 마라.',
        '일찍 일어나는 새가 벌레를 잡는다']

In [ ]:
vocab, _, vocab_size = build_vocab(data)

In [ ]:
seq_input = sentence_to_index(data, vocab)
input_length = tf.reduce_sum(tf.sign(seq_input), axis=1)
print('seq_input:')
pprint(seq_input)
print('\ninput_length:')
pprint(sess.run(input_length))

In [ ]:
embedding = tf.constant(np.random.normal(size=(vocab_size, input_size)), dtype=tf.float32)
embedded_input = tf.nn.embedding_lookup(embedding, seq_input)

In [ ]:
output, state = tf.nn.dynamic_rnn(cell, 
                                  embedded_input, 
                                  input_length, 
                                  dtype=tf.float32)

In [ ]:
print('output:')
pprint(sess.run(output))
print('\nstate:')
pprint(sess.run(state))

# Sentiment Analysis with RNN

In [ ]:
train = pd.read_csv('./data/train-5T.txt', delimiter='\t')
test = pd.read_csv('./data/test-1T.txt', delimiter='\t')

In [ ]:
X_train = train.document
Y_train = train.label
X_test = test.document
Y_test = test.label

In [ ]:
max_vocab = 50000
vocab, _, vocab_size = build_vocab(X_train, max_vocab)

In [ ]:
batches = batch_iter(list(zip(X_train, Y_train)), batch_size=64, num_epochs=15)

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.reset_default_graph()
sess = tf.Session(config=config)
model = RNN(sess=sess, vocab_size=vocab_size, lr=1e-2)
train_loss = []
train_acc = []
test_loss = []
test_acc = []

for step, batch in enumerate(batches):
    x_train, y_train = zip(*batch)
    x_train = sentence_to_index(x_train, vocab)
    acc = model.get_accuracy(x_train, y_train)
    l, _ = model.train(x_train, y_train)
    train_loss.append(l)
    train_acc.append(acc)
    
    if step % 100 == 0:
        test_batches = batch_iter(list(zip(X_test, Y_test)), batch_size=64, num_epochs=1)
        for test_batch in test_batches:
            x_test, y_test = zip(*test_batch)
            x_test = sentence_to_index(x_test, vocab)
            t_acc = model.get_accuracy(x_test, y_test)
            t_loss = model.get_loss(x_test, y_test)
            test_loss.append(t_loss)
            test_acc.append(t_acc)
        print('batch:', '%04d' % step, '\ntrain loss:', '%.5f' % np.mean(train_loss), 
              '\ttest loss:', '%.5f' % np.mean(test_loss))
        print('train accuracy:', '%.3f' % np.mean(train_acc), '\ttest accuracy:', 
              '%.3f' % np.mean(test_acc), '\n')
        train_loss = []
        train_acc = []
        test_loss = []
        test_acc = []